## qpth Julia implementation

- A solver for QP of form
 - min z'Qz/2 + q'z
 - s.t. A z = b, G z <= h

In [55]:
using LinearAlgebra;
using Test;

In [48]:
function get_sizes(G,A=Nothing)
    if ndims(G) == 2
        nineq, nz = size(G)
        nBatch = 1
    elseif ndims(G) == 3
        nBatch, nineq, nz = size(G)
    end
    
    if A != Nothing
        if length(A) > 0 && ndims(A) >= 2 
            neq = size(A)[2]
        else 
            neq = 0
        end
    else
        neq = Nothing
    end
    
    return nineq, nz, neq, nBatch
end

get_sizes (generic function with 2 methods)

In [50]:
@test get_sizes(ones(4,3), ones(2,1)) == (4,3,1,1)
@test get_sizes(ones(4,3))            == (4, 3, Nothing, 1)
@test get_sizes(ones(4,3,2))          == (3, 2, Nothing, 4)
@test get_sizes(ones(4,3,2),ones(2,1))== (3, 2, 1, 4)
@test get_sizes(ones(4,3,2),[])       == (3, 2, 0, 4)

Test Passed

In [216]:
function pre_factor_kkt(Q,G,A)
    nineq, nz, neq, _ = get_sizes(G, A)
    
    # S = [ A Q^{-1} A^T    A Q^{-1} G^T         ]
    #     [ G Q^{-1} A^T    G Q^{-1} G^T + D^{-1}]
    
    U_Q = cholesky(Q)  # NOTE: U_Q will have .U, .L fields (upper and lower traingular)
    
    U_S = zeros(neq + nineq, neq + nineq) # TODO: append .type_as(Q)
    
    G_invQ_GT = G*(inv(U_Q.L*U_Q.L')*G')
    R = G_invQ_GT
    
    if neq > 0
        invQ_AT = inv(U_Q.L*U_Q.L')*A'
        A_invQ_AT = A*invQ_AT
        G_invQ_AT = G*invQ_AT

        U11 = cholesky(A_invQ_AT)
        U12 =  U11.L' \ G_invQ_AT'
        U_S[1:neq,1:neq] = ones(neq,neq)*U11.L[1]
        U_S[1:neq,neq:neq+nineq] = ones(neq,nineq+1)*U12[1]
        R = R - U12'*U12
    end
    
    return U_Q.L, U_S, R
end 

pre_factor_kkt (generic function with 1 method)

In [217]:
Q = [[2.0,-1.0,0.0] [-1.0,2.0,-1.0] [0.0,-1.0,2.0]]
G = [1.0 2.0 1.0]
A = [2.0 0.0 -1.0]
pre_factor_kkt(Q,G,A)

([1.41421 0.0 0.0; -0.707107 1.22474 0.0; 0.0 -0.816497 1.1547], [1.65831 1.65831 1.20605 1.20605; 1.65831 1.65831 1.20605 1.20605; 1.65831 1.65831 1.20605 1.20605; 0.0 0.0 0.0 0.0], [8.54545])